In [ ]:
# import libraries

import pygeoj
import json
import pandas as pd
import numpy as np

In [ ]:
# read the source data which is named as fs
testfile = pygeoj.load("fs.geojson")

# define initial counter with size greated than estimated rows which we could get from source data site
maxlen=34000

# initialize dataframe
df = pd.DataFrame(np.full((maxlen, 2), np.nan),columns = ['counter','coordinates'])

i=0
for feature in testfile:
    y = feature.geometry.coordinates
    z = feature.properties
    string_1 = json.dumps(y)
    string_2 = json.dumps(z)
    df['counter'][i] = string_2
    df['coordinates'][i] = string_1
    i+=1
    if i%5000 == 0:
        print(i)

In [ ]:
# save the file as csv to do manual checks
df.to_csv('hitntrial.csv')

# Once the following steps are done then only only run below codes

##### 1. In excel we have 2 columns, for column 1, use formulas of substring to select Gradientpc numerical values
##### 2. Delete any observation where text goes to column C
##### 3. Delete all observations where gradientpc is not defined properly i.e. which is not a number

In [ ]:
df = pd.read_csv("hitntrial.csv")

In [ ]:
# drop non steep roads
df = df[df["Gradientpc"] >= 10]

# Derive primary key
df = df.reset_index()
df['PrimaryKey'] = df.index + 1
df['PrimaryKey'] = "F"+df.PrimaryKey.map(str)

In [ ]:
# Derive Steepness flags

df['SteepFlag'] = np.where(df['Gradientpc']>=50, 'Level 3', 'Level 2')

for i in range(len(df)):  
    if df['Gradientpc'][i] >=50:  
        df['SteepFlag'][i] = 3
    else:
        df['SteepFlag'][i] = 2
        
for i in range(len(df)):  
    if df['Gradientpc'][i] <=25:  
        df['SteepFlag'][i] = 1

# drop unwanted columns        
df = df.drop(['index','Gradientpc'], axis=1)

In [ ]:
j = df.set_index('PrimaryKey').to_json(orient='index')

In [ ]:
# Write the dataframe to the JSON file
with open("SteepNess.json", "w") as text_file:
    text_file.write("{}".format(j))